In [9]:
from newspaper import Article
from bs4 import BeautifulSoup
import requests
import urllib

#Hindu Business Line RSS Feed
url = "https://www.thehindubusinessline.com/markets/stock-markets/feeder/default.rss"

resp = requests.get(url)

bsObj = BeautifulSoup(resp.content, features = "xml")
links = bsObj.findAll('link', text=True)

##Links extracted from RSS Feed
print('EXTRACTED LINKS FROM RSS FEED')
for link in links:
    print(link.get_text())

EXTRACTED LINKS FROM RSS FEED
https://www.thehindubusinessline.com/
https://www.thehindubusinessline.com/markets/marico-buy/article25654939.ece
https://www.thehindubusinessline.com/markets/aegis-logistics-buy/article25654935.ece
https://www.thehindubusinessline.com/markets/stock-markets/investors-to-get-upto-50k-on-share-sale-in-30-min-icici-securities/article25654366.ece
https://www.thehindubusinessline.com/markets/stock-markets/china-us-trade-truce-sends-world-stocks-emerging-markets-surging/article25653998.ece
https://www.thehindubusinessline.com/markets/stock-markets/trade-war-truce-triggers-surge-in-european-shares/article25653674.ece
https://www.thehindubusinessline.com/markets/stock-markets/rcom-rallies-18-as-supreme-court-allows-spectrum-sale-to-rjio/article25653516.ece
https://premium.thehindubusinessline.com/portfolio/technical-analysis/nifty-call-consider-long-positions-on-a-rally-above-10920/article25653347.ece
https://www.thehindubusinessline.com/markets/stock-markets/asho

In [22]:
#Storing the texts in a list and using newspaper3k to get a summary

for link in links:
    article = Article(link.getText(), language="en")
    article.download()
    article.parse()
    article.nlp()
    print(article.title)
    print('-----------------------')
    print(article.text)
    print('$$$$$$$$$$$$$$$$$$$$$$$$$')
    print(article.summary)
    print('=====================')
   


The Hindu BusinessLine: Business Financial, Economy, Market, Stock
-----------------------
Click allow to get news alerts from

The Hindu BusinessLine

You can manage the notifications any time from your browser settings.
$$$$$$$$$$$$$$$$$$$$$$$$$
Click allow to get news alerts fromThe Hindu BusinessLineYou can manage the notifications any time from your browser settings.
Broker's call: Marico (Buy)
-----------------------
Chola Securities

Marico (Buy)

CMP: ₹360

Target: ₹450-480

Marico is one of India’s leading consumer products companies operating in the beauty and wellness space. Currently present in 25 countries and nurtured multiple brands in the categories of hair care, skin care, edible oils, health foods, male grooming, and fabric care. Marico’s India business markets household brands such as Parachute, Parachute Advansed, Saffola, Hair & Care, Nihar, Nihar Naturals, Livon, Set Wet, Mediker and Revive.

Market share: Coconut oils (59 per cent), Saffola — Supreme premium refi

Trade war truce triggers surge in European shares
-----------------------
Miners, autos, tech, and oil stocks all surged on Monday, driving Europe's main benchmarks up strongly after US and Chinese leaders agreed a temporary truce in an ongoing trade war which has roiled global financial markets.

Germany’s DAX the most sensitive to China and trade war fears led the way with a 2.5 per cent rise, hitting its highest level since November 14. The pan-European STOXX 600 climbed 1.9 per cent, on track for its strongest day in eight months.

US President Donald Trump and Chinese President Xi Jinping brokered a trade war truce at Saturday’s G20 dinner, agreeing to halt additional tariffs with fresh talks aimed at reaching an agreement within 90 days. Financials were the biggest boost to European shares as China-exposed bank HSBC rose and lenders across the region cheered the prospect of a detente in a trade war, which has dented world economic growth prospects.

Mining stocks led gains with a

Nikkei hits 7-week high as China-US truce lifts cyclical stocks
-----------------------
Japan's Nikkei surged to a seven-week high on Monday morning after the United States and China had during the weekend suspended new tariffs and agreed to try reaching a trade deal within 90 days, which lifted global cyclical stocks.

The Nikkei share average soared 1.4 per cent to 22,647.29 in mid-morning trade, after touching 22,677.29, the highest level since October 18.

The White House said President Donald Trump told China's President Xi Jinping at the G20 talks in Argentina that he would not boost tariffs on $200 billion of Chinese goods to 25 per cent on January 1, as previously planned.

Japanese cyclical stocks such as those in tech, autos and machinery that have large exposure to the Chinese market outperformed. Hitachi Ltd and TDK Corp surged 5 per cent, and Toyota Motor Corp advanced 3.5 per cent. Honda Motor added 2 per cent, Fanuc Corp 2.5 per cent and Yaskawa Electric 3.1 per cent.

T

In [11]:
#Using pagerank and textrank to get the summary

from nltk.corpus import brown, stopwords
from nltk.cluster.util import cosine_distance

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [29]:
import numpy as np
from nltk.tokenize import sent_tokenize
from operator import itemgetter 

#Store all text in list
article_text = []

for link in links:
    article = Article(link.getText(), language="en")
    article.download()
    article.parse()
   # print(article.text)
    article_text.append(article.text)
    
article_text.pop(0)
stop_words = stopwords.words('english')

def build_similarity_matrix(sentences, stopwords=None):
    
    # Create an empty similarity matrix
    S = np.zeros((len(sentences), len(sentences)))
 
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
 
            S[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
 
    # normalize the matrix row-wise
    for idx in range(len(S)):
        S[idx] /= S[idx].sum()
 
    return S

def pagerank(A, eps=0.0001, d=0.85):
    P = np.ones(len(A)) / len(A)
    while True:
        new_P = np.ones(len(A)) * (1 - d) / len(A) + d * A.T.dot(P)
        delta = abs(new_P - P).sum()
        if delta <= eps:
            return new_P
        P = new_P



for texts in article_text:
    sentences = sent_tokenize(texts)
    S = build_similarity_matrix(sentences, stop_words)    
    #print(S)
    sentence_ranks = pagerank(S)
    ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
    SUMMARY_SIZE = 3
    SELECTED_SENTENCES = sorted(ranked_sentence_indexes[:SUMMARY_SIZE])
    summary = itemgetter(*SELECTED_SENTENCES)(sentences)
    # Print the actual summary
    print('\nSUMMARY')
    print('------------------------------')
    for sentence in summary:
        print(''.join(sentence))
    


SUMMARY
------------------------------
Currently present in 25 countries and nurtured multiple brands in the categories of hair care, skin care, edible oils, health foods, male grooming, and fabric care.
Market share: Coconut oils (59 per cent), Saffola — Supreme premium refined oils in consumer packs (69 per cent), value added hair oils(VAHO) (34 per cent), post wash leave on serums (82 per cent), hair creams/ gel (63 per cent), saffola oats(28 per cent).
Technical rationale: a) Stock is in long-term uptrend from ₹250 levels b) In short term it corrected from ₹390 levels and got support at ₹290; c)We expect in short term price will continue its long-term uptrend towards 161.8 per cent Fibonacci at ₹450 levels; d) ₹290 levels will hold as medium-term support; e) Buy at ₹360 levels for a target of ₹450-480 levels with ₹320 as stop-loss.

SUMMARY
------------------------------
CD Equisearch

Aegis Logistics (Buy)

CMP: ₹209.75

Target: ₹454

According to the Economic Survey of India 201